In [1]:
%store

Stored variables and their in-db values:
__importClassification             -> IPython.macro.Macro(u"# Handle table-like data and
__importRegression                 -> IPython.macro.Macro(u"# Handle table-like data and


In [2]:
%store -r __importRegression

In [3]:
__importRegression

/home/maviator/anaconda2/lib/python2.7/site-packages/sklearn/cross_validation.py:44: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


In [4]:
train = pd.read_csv("../input/validation/train.csv")
test = pd.read_csv("../input/validation/test.csv")
items = pd.read_csv("../input/items.csv")
categories = pd.read_csv("../input/item_categories.csv")

print (train.shape, test.shape, items.shape, categories.shape)

((2882335, 6), (31531, 4), (22170, 3), (84, 2))


In [5]:
labels = test.pop('item_cnt_day')

In [6]:
train.head()

,date,date_block_num,shop_id,item_id,item_price,item_cnt_day
0,02.01.2013,0,59,22154,999.00,1.0
1,03.01.2013,0,25,2552,899.00,1.0
2,05.01.2013,0,25,2552,899.00,-1.0
3,06.01.2013,0,25,2554,1709.05,1.0
4,15.01.2013,0,25,2555,1099.00,1.0


In [7]:
test.head()

,Unnamed: 0,shop_id,item_id
0,0,2,31
1,1,2,486
2,2,2,787
3,3,2,794
4,4,2,968


Let's implement last month sales and check if it is comparable on error on the leaderboard
- Last month in the data is 33 (original train) and 32 (validation train)

In [8]:
last_month_sales = train.loc[train.date_block_num == 32]
last_month_sales.reset_index(inplace=True)
print last_month_sales.shape
last_month_sales.head()

(50588, 7)


,index,date,date_block_num,shop_id,item_id,item_price,item_cnt_day
0,2831747,10.09.2015,32,42,11170,58.0,1.0
1,2831748,06.09.2015,32,42,11215,349.0,1.0
2,2831749,10.09.2015,32,42,11215,349.0,1.0
3,2831750,22.09.2015,32,42,11215,349.0,1.0
4,2831751,28.09.2015,32,42,11232,299.0,1.0


In [9]:
last_month_sales.pop('date')
last_month_sales.pop('date_block_num')
last_month_sales.pop('item_price')
last_month_sales.pop('index')
last_month_sales.head()

,shop_id,item_id,item_cnt_day
0,42,11170,1.0
1,42,11215,1.0
2,42,11215,1.0
3,42,11215,1.0
4,42,11232,1.0


In [10]:
last_month_sales.item_cnt_day.describe()

count    50588.000000
mean         1.439926
std          6.132888
min         -2.000000
25%          1.000000
50%          1.000000
75%          1.000000
max        669.000000
Name: item_cnt_day, dtype: float64

In [11]:
last_month_sales = last_month_sales.groupby(['shop_id', 'item_id'], as_index=False).agg({'item_cnt_day':sum})

In [12]:
print last_month_sales.shape
last_month_sales.head()

(29678, 3)


,shop_id,item_id,item_cnt_day
0,2,33,1.0
1,2,486,1.0
2,2,792,1.0
3,2,975,1.0
4,2,1090,1.0


In [13]:
last_month_sales.item_cnt_day.describe()

count    29678.000000
mean         2.454444
std         18.446954
min         -1.000000
25%          1.000000
50%          1.000000
75%          2.000000
max       1644.000000
Name: item_cnt_day, dtype: float64

In [14]:
test.head()

,Unnamed: 0,shop_id,item_id
0,0,2,31
1,1,2,486
2,2,2,787
3,3,2,794
4,4,2,968


In [15]:
predictions = np.zeros(shape=test.shape[0])
print predictions.shape

(31531,)


In [16]:
print test.shape[0]
for i in range(test.shape[0]):
    #id0 = test['ID'][i]
    shop_id0 = test['shop_id'][i]
    item_id0 = test['item_id'][i]
    #print id0, shop_id0, item_id0
    if shop_id0 in last_month_sales.shop_id.values:
        possibles = last_month_sales.loc[last_month_sales.shop_id == shop_id0]
        if item_id0 in possibles.item_id.values: 
            t = possibles.loc[possibles.item_id==item_id0]
            predictions[i] = t.item_cnt_day.values[0]
        else:
            predictions[i] = 0.0
    else:
        predictions[i] = 0.0

31531


In [18]:
labels.shape

(31531,)

Clipping predictions in the [0, 20] range

In [19]:
for i in range(len(predictions)):
    if predictions[i] > 20:
        predictions[i] = 20
    if predictions[i] < 0:
        predictions[i] = 20

Compute RMSE

In [20]:
np.sqrt(mean_squared_error(labels, predictions))

2.9043782068433628

## Notes

- Validation set is the last month in train data
- Correct validation scheme is set with RMSE calculation
- Clipping values in the [0 20] range

- Method 0: Use last month sales as predictions, RMSE: 2.904